In [1]:
from os import environ
from citrination_client import CitrinationClient
from citrination_client import *

In [2]:
client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')
dataset_id = '151803'

In [22]:
value_query = FieldQuery(extract_as="Fracture Toughness", extract_all=True)
property_query = PropertyQuery(name=FieldQuery(filter=[Filter(equal="Fracture Toughness")]), value=value_query)
formula_query = ChemicalFieldQuery(extract_as="formula")
system_query = PifSystemQuery(chemical_formula=formula_query, properties=property_query)
dataset_query = DatasetQuery(id=[Filter(equal=dataset_id)])
data_query = DataQuery(dataset=dataset_query, system=system_query)
pif_query = PifSystemReturningQuery(size=5000, random_results=True, query=data_query)

search_result = client.search.pif_search(pif_query)

print("We found {} records".format(len(search_result.hits)))
print([x.extracted for x in search_result.hits[0:2]])

We found 396 records
[{'formula': 'B4C.xZrO2', 'Fracture toughness': ['2.52+-0.23', '2.57+-0.19', '2.71+-0.19', '3.07+-0.23', '3.25+-0.26']}, {'formula': 'Si3N4.yY2O3.zAl2O3', 'Fracture toughness': ['4.8', '5.3', '5.8', '5.9', '5.9', '6.2', '6.2', '6.2', '6.2', '6.2', '6.3', '6.3', '6.3', '6.2', '6.2', '6.3']}]


In [49]:
from pypif import pif
import csv

rows = []
pif_records = [x.system for x in search_result.hits]
for system in pif_records:
    if "x" not in system.chemical_formula and "." not in system.chemical_formula:
        for prop in system.properties:
            if prop.name == "Fracture Toughness" and prop.units == "MPa m$^{1/2}$":
                for cond in prop.conditions:
                    if cond.name == "Temperature":
                        if len(prop.scalars) == len(cond.scalars):
                            for prop_sca, cond_sca in zip(prop.scalars, cond.scalars):
                                row = [system.chemical_formula, prop_sca.value, cond_sca.value, system.references[0].citation]
                                rows.append(row)


with open('fracture_toughness.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Formula', 'Fracture Toughness (Mpa m^(1/2))', "Temperature (degC), Reference"])
    writer.writerows(rows)
    